# ABS Quarterly Dwelling Stock 6432

## Python set-up

In [1]:
# system imports
from functools import cache
from typing import TypedDict

In [2]:
# analytic imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# local imports
import abs_data_capture as adc
import plotting as p
import rba_data_capture as rba

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get core data from ABS

In [4]:
def get_data() -> tuple[dict[str, pd.DataFrame], pd.DataFrame, str]:
    """Capture relevant ABS data, set-up plotting output directories"""

    landing_page_ = adc.AbsLandingPage(
        theme="economy",
        parent_topic="price-indexes-and-inflation",
        topic="total-value-dwellings",
    )
    abs_dict_ = adc.get_abs_data(landing_page_)
    source_, chart_dir, _cat_id, meta_ = adc.get_fs_constants(abs_dict_, landing_page_)

    # plotting set-up
    p.clear_chart_dir(chart_dir)
    p.set_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")

    return abs_dict_, meta_, source_

In [5]:
abs_dict, meta, source = get_data()

# let's get a quick look at the timeliness of the latest data
print(abs_dict["1"].tail(1).index)

A little unexpected: We need to fake up a zip file
PeriodIndex(['2023Q4'], dtype='period[Q-DEC]', name='Series ID')


## Plot the data

In [6]:
def headline() -> None:
    """Headline charts"""

    plotable = [
        "Value of dwelling stock; Owned by Households ;  Australia ;",
        "Value of dwelling stock; Owned by All Sectors ;  Australia ;",
        "Value of dwelling stock; Owned by Non-Households ;  Australia ;",
        "Mean price of residential dwellings ;  Australia ;",
        "Number of residential dwellings ;  Australia ;",
    ]
    data = abs_dict["1"]
    for item in plotable:
        search = {item: adc.metacol.did}
        series_id, units = adc.find_id(meta, search)
        series, units = p.recalibrate_series(data[series_id], units, verbose=True)
        stype = meta[meta[adc.metacol.id] == series_id][adc.metacol.stype].values[0]

        ax = series.plot()
        p.finalise_plot(
            ax,
            title=item.rsplit(";", maxsplit=2)[0],
            ylabel=units,
            rfooter=source,
            lfooter=f"Australia. {stype} series. ",
            show=SHOW,
        )

In [7]:
headline()

recalibrate(): No adjustments needed


## Calculate and plot population per dwelling

In [8]:
def get_population() -> tuple[pd.Series, str]:
    """Get latest population estimates from national accounts."""

    landing_page = adc.AbsLandingPage(
        theme="economy",
        parent_topic="national-accounts",
        topic="australian-national-accounts-national-income-expenditure-and-product",
    )
    gdp_dict = adc.get_abs_data(landing_page)
    _, _, cat_id, gdp_meta = adc.get_fs_constants(gdp_dict, landing_page)

    gdp_did = "Gross domestic product"
    gdp_pc_did = "GDP per capita"
    table = "1"
    data = pd.DataFrame()
    for item in gdp_did, gdp_pc_did:
        search = {
            item: adc.metacol.did,
            "Chain volume measures": adc.metacol.did,
            "$": adc.metacol.unit,
            "Original": adc.metacol.stype,
            table: adc.metacol.table,
        }
        series_id, _unit = adc.find_id(gdp_meta, search, verbose=False)
        data[item] = gdp_dict[table][series_id]
    population = data[data.columns[0]] / data[data.columns[1]] * 1_000_000

    return population, cat_id

In [9]:
def pop_per_dwelling() -> None:
    """Plot population per dwelling."""

    population, cat_gdp = get_population()

    search = {"Number of residential dwellings ;  Australia ;": adc.metacol.did}
    series_id, units = adc.find_id(meta, search)
    assert units == "Thousands"
    dwellings = abs_dict["1"][series_id] * 1_000
    pop_per_dwellings = (population / dwellings).dropna()

    ax = pop_per_dwellings.plot()
    p.finalise_plot(
        ax,
        title="Implicit population per dwelling",
        ylabel="Persons",
        rfooter=f"{source} {cat_gdp}",
        lfooter="Australia. Original series. ",
        show=SHOW,
    )

In [10]:
pop_per_dwelling()

## Calculate and plot mean dwelling value per FT annual ordinary earnings 

In [11]:
def get_mean_value() -> pd.Series:

    search = {"Mean price of residential dwellings ;  Australia ;": adc.metacol.did}
    series_id, units = adc.find_id(meta, search)
    assert units.strip() == "$ Thousand"
    mean_value = abs_dict["1"][series_id] * 1_000

    return mean_value

In [12]:
WEEKS_PER_YEAR = 365.25 / 7


def get_earnings() -> tuple[pd.Series, str]:
    """Return Average Annual FT Adult Ordinary Time Earnings for Persons.
    Note: This data is published biannually."""

    landing_page = adc.AbsLandingPage(
        theme="labour",
        parent_topic="earnings-and-working-conditions",
        topic="average-weekly-earnings-australia",
    )
    awe_dict = adc.get_abs_data(landing_page)
    _, _, cat_id, awe_meta = adc.get_fs_constants(awe_dict, landing_page)

    table = "3"  # Original series
    awe_did = "Earnings; Persons; Full Time; Adult; Ordinary time earnings ;"
    search = {
        awe_did: adc.metacol.did,
        table: adc.metacol.table,
    }
    series_id, _unit = adc.find_id(awe_meta, search, exact=True, verbose=False)
    annual_ft_earnings = awe_dict[table][series_id] * WEEKS_PER_YEAR
    return annual_ft_earnings, cat_id

In [13]:
def q_nov_to_dec(series: pd.Series) -> pd.Series:
    return series.to_timestamp(how="end").to_period(freq="Q-DEC")

In [14]:
def mean_dwelling_value_per_earnings() -> None:
    """Plot mean dwelling value per average annual FT ordinary time earnings."""

    earnings, cat_awe = get_earnings()
    earnings = q_nov_to_dec(earnings)
    mean_value = get_mean_value()
    value_per_earnings = (mean_value / earnings).dropna()

    ax = value_per_earnings.plot()
    p.finalise_plot(
        ax,
        title="Mean dwelling value / Annual ave FT ordinary earnings",
        ylabel="Multiples",
        rfooter=f"{source} {cat_awe}",
        lfooter="Australia. Original series. ",
        show=SHOW,
    )

In [15]:
mean_dwelling_value_per_earnings()

Caution: Could not find the 'Index' sheet in 63020do015_20234.xlsx. File not included
Caution: Could not find the 'Index' sheet in 63020do016_20234.xlsx. File not included
Caution: Could not find the 'Index' sheet in 63020do017_20234.xlsx. File not included


## Housing repayment affordability
Weekly loan repayment as a percent of weekly income

In [16]:
@cache
def get_interest_rates() -> dict[str, pd.Series]:

    table = "Indicator Lending Rates – F5"
    meta, data = rba.get_data(table)
    # display(meta.Title.to_list())
    desired = (
        "Lending rates; Housing loans; Banks; Variable; Standard; Owner-occupier",
        "Lending rates; Housing loans; Banks; Variable; Discounted; Owner-occupier",
        "Lending rates; Housing loans; Banks; 3-year fixed; Owner-occupier",
    )
    ret = {}
    for d in desired:
        column = meta[meta.Title == d]["Series ID"].values[0]
        title = d.split(";", maxsplit=2)[-1].strip()
        ret[title] = data[column]

    return ret

In [17]:
class Assumptions(TypedDict):
    loan_to_value: int  # percent
    loan_term: int  # years
    repayment_freq: float  # weeks


def calculate_repayments(
    a: Assumptions,
    dwelling_value: pd.Series,
    weekly_earnings: pd.Series,
    loan_rates: dict[str, pd.Series],
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Based on assumptions, calculate the repayments for a new loan
    at the time the loan was made. Return a tuple of DataFrames.
    The first DataFrame is weekly repayment amounts in nominal $,
    The second DataFrane is weekly repayment amounts as a % of AWE."""

    # preliminaries
    n_per_year = WEEKS_PER_YEAR / a["repayment_freq"]  # repayments per year
    n_per_term = a["loan_term"] * n_per_year  # repayments per loan term
    principal = dwelling_value * a["loan_to_value"] / 100.0

    # calculate - weekly repayments in nominal $, and as a % of AWE
    repayment_to_income = pd.DataFrame()
    weekly_repayment = pd.DataFrame()
    for label, series in loan_rates.items():
        period_rate = series / 100.00 / n_per_year  # convert percent to rate
        period_payment = (
            (period_rate * principal) / (1 - (1 / (1 + period_rate) ** n_per_term))
        ).dropna()
        weekly_payment = period_payment / a["repayment_freq"]
        weekly_repayment[label] = weekly_payment
        repayment_to_income[label] = weekly_payment / weekly_earnings * 100.0

    return weekly_repayment, repayment_to_income

In [18]:
def triangle(series: pd.Series) -> pd.DataFrame:
    """Convert a Series into a lower-left-triangle DataFrame."""

    return (
        pd.DataFrame(np.diag(series), index=series.index, columns=series.index)
        .astype(float)
        .replace(0.0, np.nan)
        .ffill()
    )

In [19]:
def repayment_affordability():
    """Produce loan repayment affordability charts."""

    # Input data
    mean_dwelling_value = get_mean_value()  # PeriodIndex = Q-DEC
    annual_earnings, cat_awe = get_earnings()  # PeriodIndex = Q-NOV
    annual_earnings = q_nov_to_dec(annual_earnings)  # PeriodIndex = Q-DEC
    weekly_earnings = (annual_earnings / WEEKS_PER_YEAR).dropna()
    orig_loan_rates = get_interest_rates()  # DatetimeIndex, monthly

    # plot - weekly FT ordinary earnings
    p.line_plot(
        weekly_earnings,
        starts=(None, pd.Period("2011-01-01", freq="Q")),
        title="Weekly FT Ordinary Earnings",
        ylabel="$ per week (nominal)",
        rfooter=f"ABS {cat_awe}",
        lfooter=f"Australia.",
        show=SHOW,
    )

    # plot - lending rates from the RBA
    indicator_rates = pd.DataFrame(orig_loan_rates).to_period(freq="M")
    indicator_rates = indicator_rates[~indicator_rates.index.duplicated(keep="last")]
    p.line_plot(
        indicator_rates,
        starts=(None, pd.Period("2011-01-01", freq="M")),
        title="Home loan rates",
        ylabel="Per cent per year",
        width=[1, 2, 3],
        rfooter=f"RBA F5",
        lfooter=f"Australia.",
        show=SHOW,
    )

    # convert monthly loan rate to quarterly data
    loan_rates = {}
    for label, series in orig_loan_rates.items():
        series = series.to_period(freq="Q-DEC")
        series = series.groupby(series.index).last()
        loan_rates[label] = series

    # --- calculate what a new loan would cost, subject to assumptions
    assumptions: Assumptions = {
        "loan_to_value": 80,  # percent
        "loan_term": 30,  # years
        "repayment_freq": 2.0,  # weeks
    }
    assumptions_text = (
        f"Assumptions: LVR={assumptions['loan_to_value']}% of mean dwelling value, "
        f"repayment period={assumptions['repayment_freq']}-weeks, "
        f"loan-term={assumptions['loan_term']}-years. "
    )

    # Calculate
    weekly_repayment, repayment_to_income = calculate_repayments(
        assumptions, mean_dwelling_value, weekly_earnings, loan_rates
    )

    # plot - weekly repayments in nominal $
    p.line_plot(
        weekly_repayment,
        title="New home loan repayments (per week)",
        width=[1, 2, 3],
        ylabel="$ per week",
        rfooter=f"{source} RBA F5",
        lfooter=f"Australia. {assumptions_text}",
        show=SHOW,
    )

    # plot - repayments as a % of average ordinary full-time ordinary earnings
    p.line_plot(
        repayment_to_income.dropna(how="all"),
        title="New home loan repayments / Ave FT ordinary earnings",
        ylabel="Per cent",
        width=[1, 2, 3],
        rfooter=f"{source} {cat_awe} RBA F5",
        lfooter=f"Australia. {assumptions_text}",
        show=SHOW,
    )

    # ---- Bill Shock
    # let's think about bill-shock with a three year
    # fixed term loan followed by discount variable rate.

    # create starting point dwelling_values (assumed purchase price)
    v = mean_dwelling_value
    purchase_price = triangle(v)
    nominal = pd.DataFrame(index=v.index)
    standardised = pd.DataFrame(index=v.index)
    standardised = standardised[standardised.index.quarter.isin((2, 4))]

    # create fixed rates
    fixed_col = "Banks; 3-year fixed; Owner-occupier"
    fr = loan_rates[fixed_col].dropna()
    fr = fr[fr.index >= purchase_price.index[0]].copy()
    fixed_rates = triangle(fr)

    # calculate payment regime
    fixed_years = 3
    var_col = "Banks; Variable; Discounted; Owner-occupier"
    for col in purchase_price.columns:
        lr = {fixed_col: fixed_rates[col], var_col: loan_rates[var_col].dropna()}
        nom, stdz = calculate_repayments(
            assumptions, purchase_price[col], weekly_earnings, lr
        )

        fixed_period = pd.period_range(start=col, periods=fixed_years * 4).intersection(
            purchase_price.index
        )
        nominal.loc[fixed_period, col] = nom.loc[fixed_period, fixed_col]
        nominal[col] = nominal[col].where(nominal[col].notna(), other=nom[var_col])

        if col not in standardised.index:
            continue

        fixed_period = fixed_period.intersection(standardised.index)
        standardised.loc[fixed_period, col] = stdz.loc[fixed_period, fixed_col]
        standardised[col] = standardised[col].where(
            standardised[col].notna(), other=stdz[var_col]
        )

    # - plot bill-shock in nominal $
    colors = [
        mpl.colors.rgb2hex(x)
        for x in plt.cm.jet(np.linspace(0, 1, len(nominal.columns)))
    ]
    p.line_plot(
        nominal,
        title=f"Repayments: {fixed_years}-years fixed then discount variable rate",
        ylabel="$ per week (nominal)",
        rfooter=f"{source} RBA F5",
        lfooter=f"Australia. {assumptions_text}",
        color=colors,
        legend=False,
        show=SHOW,
    )

    # - plot bill-shock in % AWE terms
    colors = [
        mpl.colors.rgb2hex(x)
        for x in plt.cm.jet(np.linspace(0, 1, len(standardised.columns)))
    ]
    p.line_plot(
        standardised,
        title=f"Repayments: {fixed_years}-years fixed then discount variable rate",
        ylabel="% FT Ordinary Earnings",
        rfooter=f"{source} {cat_awe} RBA F5",
        lfooter=f"Australia. {assumptions_text}",
        tags="standardized",
        color=colors,
        legend=False,
        show=SHOW,
    )

In [20]:
repayment_affordability()

## Finished

In [21]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Mar 17 2024 20:45:35

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

numpy     : 1.26.4
matplotlib: 3.8.3
pandas    : 2.2.1

Watermark: 2.4.3



In [22]:
print("Finished")

Finished
